In [ ]:
from env.core import FunctionExpr, Param, AndNode, OrNode, IntBooleanNode
from env.full_state import FullState
from env.meta_env import MetaInfo
from env.goal_env import GoalEnv
from env.node_types import HaveScratch

params = (Param.from_int(1), Param.from_int(2), Param.from_int(3))
p1, p2, p3 = params
goal = HaveScratch.with_goal(
    FunctionExpr.with_child(
        OrNode(
            AndNode(p1, p2, IntBooleanNode(1)),
            AndNode(p2, p3),
        ),
    )
)

env = GoalEnv(goal)
node_types = env.full_state.node_types()
print('Amount of node types:', len(node_types))

In [2]:
# selected_types = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_all_types)).apply()
# GoalEnv.symbolic(selected_types, node_types)

In [3]:
selected_goal = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_goal)).apply()
assert selected_goal == goal

In [ ]:
GoalEnv.symbolic(selected_goal, node_types)

In [ ]:
GoalEnv.data_array(selected_goal, node_types)

In [ ]:
env.to_symbolic_current_state()

In [ ]:
env.to_data_array_current_state()

In [ ]:
len(env.to_data_array())

In [9]:
# details = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_all_types_details)).apply()
# selected = details.nested_arg(102).apply()
# GoalEnv.symbolic(selected, node_types)

In [10]:
# import sympy
# sympy.latex(GoalEnv.symbolic(selected, node_types))

In [11]:
# GoalEnv.data_array(selected, node_types)

In [12]:
# env.to_symbolic()

In [13]:
env.full_state.validate()

In [ ]:
from env.full_state import HistoryNode
from env.state import State
def get_current_state(env: GoalEnv):
    return env.full_state.nested_args((FullState.idx_current, HistoryNode.idx_state)).apply().cast(State)
current_state = get_current_state(env)
GoalEnv.symbolic(current_state, node_types)

In [15]:
from env.state import State
assert current_state == State.create()

In [ ]:
from env.action_impl import CreateScratch
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
GoalEnv.symbolic(current_state, node_types)

In [17]:
assert current_state == State.from_raw(
    scratchs=tuple([None]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.full_state import HistoryGroupNode
from env.core import Optional
def get_last_history_meta(env: GoalEnv):
    history = env.full_state.history.apply().cast(HistoryGroupNode)
    last = history.as_tuple[-1]
    return last.meta_data.apply().nested_arg(Optional.idx_value).apply()
last_history_meta = get_last_history_meta(env)
GoalEnv.symbolic(last_history_meta, node_types)

In [19]:
from env.action import ActionData
from env.action_impl import CreateScratchOutput
from env.state import StateScratchIndex

assert last_history_meta == ActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(1),
        Optional(),
    )),
    exception=Optional(),
)

In [20]:
# GoalEnv.symbolic(selected, node_types)

In [21]:
# GoalEnv.symbolic(env.full_state.history.apply(), node_types)

In [22]:
# selected_types = env.full_state.nested_args((FullState.idx_meta, MetaInfo.idx_from_int_group)).apply()
# GoalEnv.symbolic(selected_types, node_types)

In [ ]:
from env.action_impl import DefineScratchFromInt
# should return an exception (index won't return element)
action = DefineScratchFromInt.from_raw(1, len(node_types)+1, 3)
env.step(action)
current_state = get_current_state(env)
GoalEnv.symbolic(current_state, node_types)

In [24]:
assert current_state == State.from_raw(
    scratchs=tuple([None]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
last_history_meta = get_last_history_meta(env)
GoalEnv.symbolic(last_history_meta, node_types)

In [26]:
from env.action import IAction, ActionData
from env.core import BooleanExceptionInfo, IsEmpty

def get_empty_exception(action: IAction):
    return ActionData.from_args(
        action=Optional(action),
        output=Optional(),
        exception=Optional(BooleanExceptionInfo(IsEmpty(Optional()))),
    )

assert last_history_meta == get_empty_exception(action)

In [ ]:
from env.action_impl import DefineScratchFromIntIndex
from env.core import INode
from env.meta_env import SubtypeOuterGroup, GeneralTypeGroup
from env.full_state import MetaAllTypesTypeIndex

def get_info_type_index(node_type: type[INode], env: GoalEnv, node_types: tuple[type[INode], ...]):
    index_type_idx = node_types.index(MetaAllTypesTypeIndex) + 1
    type_node = node_types[index_type_idx-1].as_type()
    selected_types = env.full_state.meta.apply().nested_args((
        MetaInfo.idx_full_state_int_index_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(type_node) + 1
    node_idx = node_types.index(node_type) + 1
    return meta_idx, node_idx

meta_idx, type_idx = get_info_type_index(AndNode, env, node_types)
action = DefineScratchFromIntIndex.from_raw(1, meta_idx, type_idx)
env.step(action)
current_state = get_current_state(env)
GoalEnv.symbolic(current_state, node_types)

In [28]:
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type()]),
    args_groups=tuple(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
last_history_meta = get_last_history_meta(env)
GoalEnv.symbolic(last_history_meta, node_types)

In [30]:
from env.core import Integer
from env.full_state import MetaFullStateIntIndexTypeIndex
from env.action_impl import DefineScratchOutput
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromIntIndex(
        StateScratchIndex(1),
        MetaFullStateIntIndexTypeIndex(meta_idx),
        Integer(type_idx)
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Optional(AndNode.as_type()),
    )),
    exception=Optional(),
)

In [31]:
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [32]:
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), None]),
    args_groups=tuple(),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(2),
        Optional(),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [33]:
meta_idx, type_idx = get_info_type_index(OrNode, env, node_types)
action = DefineScratchFromIntIndex.from_raw(2, meta_idx, type_idx)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [34]:
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type()]),
    args_groups=tuple(),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromIntIndex(
        StateScratchIndex(2),
        MetaFullStateIntIndexTypeIndex(meta_idx),
        Integer(type_idx)
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(2),
        Optional(OrNode.as_type()),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [35]:
action = CreateScratch.create()
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [36]:
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), None]),
    args_groups=tuple(),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(CreateScratch(Optional())),
    output=Optional(CreateScratchOutput(
        StateScratchIndex(3),
        Optional(),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
from env.action_impl import CreateArgsGroup
action = CreateArgsGroup.from_raw(3, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)
GoalEnv.symbolic(current_state, node_types)

In [38]:
from env.core import (
    ExtendedTypeGroup,
    RestTypeGroup,
    UnknownType,
    LaxOpaqueScope,
    OptionalValueGroup,
    ScopeId,
)
from env.state import PartialArgsGroup, StateArgsGroupIndex
from env.action_impl import CreateArgsGroupOutput

arg_group = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(),
            Optional(),
            Optional(),
        ),
    )
)

assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), None]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(CreateArgsGroup(
        Integer(3),
        Optional(),
        Optional(),
    )),
    output=Optional(CreateArgsGroupOutput(
        StateArgsGroupIndex(1),
        arg_group,
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [39]:
from env.action_impl import DefineScratchFromInt

def get_from_int_type_index(node_type: type[INode], env: GoalEnv):
    selected_types = env.full_state.meta.apply().nested_args((
        MetaInfo.idx_from_int_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(node_type.as_type()) + 1
    return meta_idx

meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [40]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(1),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(Param.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [41]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [42]:
from env.core import NodeArgIndex
from env.state import OptionalContext
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(1)),
            Optional(),
            Optional(),
        ),
    )
)

assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(1),
        OptionalContext(Param.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [43]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 2)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [44]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(2)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(2),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [45]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 2, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [46]:
from env.core import NodeArgIndex
from env.state import OptionalContext
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(1)),
            Optional(Param.from_int(2)),
            Optional(),
        ),
    )
)

assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(2)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(2),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(2),
        OptionalContext(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [47]:
meta_idx = get_from_int_type_index(IntBooleanNode, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [48]:
from env.full_state import MetaFromIntTypeIndex
assert current_state == State.from_raw(
    scratchs=tuple([AndNode.as_type(), OrNode.as_type(), IntBooleanNode.from_int(1)]),
    args_groups=tuple([arg_group]),
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(1),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(IntBooleanNode.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [49]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(1, 3, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [50]:
from env.core import NodeArgIndex
from env.state import OptionalContext
from env.action_impl import DefineArgsGroupArgOutput

arg_group = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(1)),
            Optional(Param.from_int(2)),
            Optional(IntBooleanNode.from_int(1)),
        ),
    )
)

scratchs = tuple([AndNode.as_type(), OrNode.as_type(), IntBooleanNode.from_int(1)])
args_groups = tuple([arg_group])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(1),
        NodeArgIndex(3),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(1),
        NodeArgIndex(3),
        OptionalContext(IntBooleanNode.from_int(1)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [51]:
from env.action_impl import CreateArgsGroup
action = CreateArgsGroup.from_raw(2, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [52]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(),
            Optional(),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(CreateArgsGroup(
        Integer(2),
        Optional(),
        Optional(),
    )),
    output=Optional(CreateArgsGroupOutput(
        StateArgsGroupIndex(2),
        arg_group_2,
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [53]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 2)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [54]:
scratchs = tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(2)])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(2),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [55]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [56]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(2)),
            Optional(),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        OptionalContext(Param.from_int(2)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [57]:
meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(3, meta_idx, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [58]:
scratchs = tuple([AndNode.as_type(), OrNode.as_type(), Param.from_int(3)])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromInt(
        StateScratchIndex(3),
        MetaFromIntTypeIndex(meta_idx),
        Integer(3),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(Param.from_int(3)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [59]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 2, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [60]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(Param.from_int(2)),
            Optional(Param.from_int(3)),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        OptionalContext(Param.from_int(3)),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [61]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(3, 1, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [62]:
call_1 = AndNode(
    Param.from_int(1),
    Param.from_int(2),
    IntBooleanNode.from_int(1),
)
scratchs = tuple([AndNode.as_type(), OrNode.as_type(), call_1])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(3),
        StateScratchIndex(1),
        Optional(StateArgsGroupIndex(1)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(3),
        Optional(call_1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [63]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(1, 1, 2)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [64]:
call_2 = AndNode(
    Param.from_int(2),
    Param.from_int(3),
)
scratchs = tuple([call_2, OrNode.as_type(), call_1])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(1),
        StateScratchIndex(1),
        Optional(StateArgsGroupIndex(2)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Optional(call_2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [65]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 1, 3)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [66]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(call_1),
            Optional(Param.from_int(3)),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        StateScratchIndex(3),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(1),
        OptionalContext(call_1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [67]:
from env.action_impl import DefineArgsGroup
action = DefineArgsGroup.from_raw(2, 2, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [68]:
arg_group_2 = PartialArgsGroup(
    ExtendedTypeGroup(RestTypeGroup(UnknownType())),
    LaxOpaqueScope(
        ScopeId(1),
        OptionalValueGroup(
            Optional(call_1),
            Optional(call_2),
        ),
    )
)

args_groups = tuple([arg_group, arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineArgsGroup(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        StateScratchIndex(1),
    )),
    output=Optional(DefineArgsGroupArgOutput(
        StateArgsGroupIndex(2),
        NodeArgIndex(2),
        OptionalContext(call_2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [69]:
from env.action_impl import DefineScratchFromFunctionWithArgs
action = DefineScratchFromFunctionWithArgs.from_raw(1, 2, 2)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [70]:
call_3 = OrNode(
    call_1,
    call_2,
)
scratchs = tuple([call_3, OrNode.as_type(), call_1])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DefineScratchFromFunctionWithArgs(
        StateScratchIndex(1),
        StateScratchIndex(2),
        Optional(StateArgsGroupIndex(2)),
    )),
    output=Optional(DefineScratchOutput(
        StateScratchIndex(1),
        Optional(call_3),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [ ]:
GoalEnv.symbolic(current_state, node_types)

In [ ]:
GoalEnv.data_array(current_state, node_types)

In [73]:
from env.action_impl import DeleteArgsGroupOutput
action = DeleteArgsGroupOutput.from_raw(1, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [74]:
args_groups = tuple([arg_group_2])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    output=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [75]:
from env.action_impl import DeleteArgsGroupOutput
action = DeleteArgsGroupOutput.from_raw(1, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [76]:
args_groups = tuple()

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    output=Optional(DeleteArgsGroupOutput(
        StateArgsGroupIndex(1),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [77]:
from env.action_impl import DeleteScratchOutput
action = DeleteScratchOutput.from_raw(3, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [78]:
scratchs = tuple([call_3, OrNode.as_type()])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DeleteScratchOutput(
        StateScratchIndex(3),
    )),
    output=Optional(DeleteScratchOutput(
        StateScratchIndex(3),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [79]:
from env.action_impl import DeleteScratchOutput
action = DeleteScratchOutput.from_raw(2, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [80]:
scratchs = tuple([call_3])

assert current_state == State.from_raw(
    scratchs=scratchs,
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(DeleteScratchOutput(
        StateScratchIndex(2),
    )),
    output=Optional(DeleteScratchOutput(
        StateScratchIndex(2),
    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is False

In [81]:
from env.core import FunctionExpr
from env.action_impl import DefineScratchFromSingleArg

def get_single_child_type_index(node_type: type[INode], env: GoalEnv):
    selected_types = env.full_state.meta.apply().nested_args((
        MetaInfo.idx_single_child_group,
        SubtypeOuterGroup.idx_subtypes,
    )).apply().cast(GeneralTypeGroup)
    meta_idx = selected_types.as_tuple.index(node_type.as_type()) + 1
    return meta_idx

meta_idx = get_single_child_type_index(FunctionExpr, env)
action = DefineScratchFromSingleArg.from_raw(1, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [ ]:
GoalEnv.symbolic(current_state, node_types)

In [ ]:
GoalEnv.data_array(current_state, node_types)

In [84]:
from env.state import Scratch
content = current_state.nested_args((
    State.idx_scratch_group,
    1,
    Scratch.idx_child,
    Optional.idx_value,
)).apply()
assert content == goal.definition_expr.apply()

In [ ]:
from env.state import StateScratchIndex
verification = goal.evaluate(current_state, StateScratchIndex(1))
verification.raise_on_false()
verification.as_bool

In [86]:
from env.action_impl import CreateScratch
action = CreateScratch.create()
env.step(action)

meta_idx = get_from_int_type_index(Param, env)
action = DefineScratchFromInt.from_raw(2, meta_idx, 4)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [ ]:
from env.state import StateScratchIndex
verification = goal.evaluate(current_state, StateScratchIndex(2))
verification.as_bool

In [88]:
action = DeleteScratchOutput.from_raw(2, 0, 0)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [89]:
from env.action_impl import VerifyGoal
from env.state import StateScratchIndex
meta_idx = get_from_int_type_index(StateScratchIndex, env)
action = VerifyGoal.from_raw(0, meta_idx, 1)
env.step(action)
current_state = get_current_state(env)
last_history_meta = get_last_history_meta(env)

In [ ]:
GoalEnv.symbolic(current_state, node_types)

In [ ]:
GoalEnv.symbolic(last_history_meta, node_types)

In [ ]:
from env.action_impl import VerifyGoalOutput
from env.state import StateMetaInfo, GoalAchieved

scratch_goal = FunctionExpr(ExtendedTypeGroup.rest(), call_3)

assert current_state == State.from_raw(
    meta_info=StateMetaInfo.create_with_goal(GoalAchieved.achieved()),
    scratchs=[scratch_goal],
    args_groups=args_groups,
)
assert last_history_meta == ActionData.from_args(
    action=Optional(VerifyGoal(
        Optional(),
        MetaFromIntTypeIndex(meta_idx),
        Integer(1),
    )),
    output=Optional(VerifyGoalOutput(
        Optional(),
        StateScratchIndex(1),

    )),
    exception=Optional(),
)
assert env.full_state.goal_achieved() is True